## Install packages

In [1]:
#! pip install vllm
!pip install vllm==0.3.3
!pip install flash-attn

from vllm import LLM
from vllm import SamplingParams
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 MB 19.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cac

## Load model

In [2]:
# llm = LLM(model='mistralai/Mistral-7B-Instruct-v0.1')
llm = LLM(model='TheBloke/Mistral-7B-Instruct-v0.2-AWQ')

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

WARNING 04-11 05:40:32 config.py:193] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-11 05:40:32 llm_engine.py:87] Initializing an LLM engine with config: model='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', tokenizer='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

INFO 04-11 05:40:39 weight_utils.py:163] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

INFO 04-11 05:44:47 llm_engine.py:357] # GPU blocks: 14037, # CPU blocks: 2048
INFO 04-11 05:44:50 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-11 05:44:50 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-11 05:44:57 model_runner.py:756] Graph capturing finished in 8 secs.


## Import data from google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [88]:
proj_path = "/content/drive/My Drive/ubc/COLX531/"
data_set = "eLife" #"PLOS" #

data_type = "val_mini" #"test" # "val" #
model_name = "mistral_7b_awq"
method = "abstract_keyword_fewshot"
prompt_name = "5shots_with200to300words"

write_path = f'{proj_path}{model_name}/{method}/{data_type}/{prompt_name}/'

In [89]:
print(write_path)

/content/drive/My Drive/ubc/COLX531/mistral_7b_awq/abstract_keyword_fewshot/val_mini/5shots_with200to300words/


In [90]:
def loader(path, sections):
    with open(path, 'r') as f:
        df = pd.read_json(path,
                    orient="records",
                    lines=True)
        return df[sections]

In [91]:
sections = ['lay_summary', 'abstract', 'introduction', 'keywords']

path = f'{proj_path}data/{data_set}_train_mini.jsonl'
train_df = loader(path, sections)

path = f'{proj_path}data/{data_set}_{data_type}.jsonl'
input_df = loader(path, sections)

In [92]:
train_df.iloc[0]

lay_summary     In nature , plant growth is often limited by u...
abstract        Plants as sessile organisms can adapt to envir...
introduction     Abiotic stress is a major threat to global cr...
keywords         [chromosomes and gene expression, plant biology]
Name: 0, dtype: object

In [93]:
input_df.iloc[0]

lay_summary     Forgetting is a common experience in our every...
abstract        Previous studies tracking AMPA receptor ( AMPA...
introduction     A major goal of cellular and molecular neuros...
keywords        [structural biology and molecular biophysics, ...
Name: 0, dtype: object

In [94]:
len(input_df)

24

## Model

In [95]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.05, max_tokens=1024)

In [96]:
import random
random.seed(531)

number_of_shot = 5
shot_idx = random.sample(train_df.index.tolist(), number_of_shot)
shot_df = train_df.iloc[shot_idx]

In [97]:
prompt = '[INST] Write a lay summary with 200 to 300 words, put the summary in 1 paragraph. '
for idx in shot_idx:
    text = f"Abstract: '{train_df.iloc[idx].abstract}', Introduction: '{train_df.iloc[idx].introduction}', Keyword: '{train_df.iloc[idx].keywords}', Lay Summary: '{train_df.iloc[idx].lay_summary}'; "
    prompt += text

prompt += f"Abstract: '{input_df.iloc[0].abstract}', Introduction: '{input_df.iloc[0].introduction}', Keyword: '{input_df.iloc[0].keywords}', Lay Summary: [/INST]"

In [98]:
prompt

"[INST] Write a lay summary with 200 to 300 words, put the summary in 1 paragraph. Abstract: 'Faced with potential harm , individuals must estimate the probability of threat and initiate an appropriate fear response . In the prevailing view , threat probability estimates are relayed to the ventrolateral periaqueductal gray ( vlPAG ) to organize fear output . A straightforward prediction is that vlPAG single-unit activity reflects fear output , invariant of threat probability . We recorded vlPAG single-unit activity in male , Long Evans rats undergoing fear discrimination . Three 10 s auditory cues predicted unique foot shock probabilities: danger ( p=1 . 00 ) , uncertainty ( p=0 . 375 ) and safety ( p=0 . 00 ) . Fear output was measured by suppression of reward seeking over the entire cue and in one-second cue intervals . Cued fear non-linearly scaled to threat probability and cue-responsive vlPAG single-units scaled their firing on one of two timescales: at onset or ramping toward sho

In [99]:
output = llm.generate(prompt, sampling_params)
answer = output[0].outputs[0].text.strip('\n').replace('\n', '')+'\n'

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.14s/it]


In [100]:
len(prompt.split())

10299

In [101]:
def generate_prompt(train_df, input_df, number_of_shot):
    prompts = []
    for i in range(len(input_df)):
        shot_idx = random.sample(train_df.index.tolist(), number_of_shot)
        shot_df = train_df.iloc[shot_idx]
        prompt = '[INST] Write a lay summary with 200 to 300 words, put the summary in 1 paragraph. '
        for idx in shot_idx:
            text = f"Abstract: '{train_df.iloc[idx].abstract}', Introduction: '{train_df.iloc[idx].introduction}', Keyword: '{train_df.iloc[idx].keywords}', Lay Summary: '{train_df.iloc[idx].lay_summary}'; "
            prompt += text

        prompt += f"Abstract: '{input_df.iloc[i].abstract}', Introduction: '{input_df.iloc[i].introduction}', Keyword: '{input_df.iloc[i].keywords}', Lay Summary: [/INST]"
        prompts.append(prompt)
    return prompts

In [102]:
prompts = generate_prompt(train_df, input_df, number_of_shot)

In [103]:
len(prompts)

24

In [104]:
output = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 24/24 [00:54<00:00,  2.29s/it]


In [105]:
print(write_path + data_set.lower() + ".txt")

/content/drive/My Drive/ubc/COLX531/mistral_7b_awq/abstract_keyword_fewshot/val_mini/5shots_with200to300words/elife.txt


In [106]:
with open(write_path + data_set.lower() + ".txt", 'w') as f:
    for item in output:
        f.write(item.outputs[0].text.strip('\n').replace('\n', '')+'\n')